In [1]:
import yfinance as yf
import talib
import pandas as pd
from backtesting import Backtest, Strategy
import numpy as np

C:\Users\Aryan jain\Desktop\programs\CFA\bt\Lib\site-packages\backtesting\_plotting.py:55: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support, such as old IDEs. Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
sectors = {
    "IT": "^CNXIT",
    "Bank": "^NSEBANK",
    "Pharma": "^CNXPHARMA",
    "FMCG": "^CNXFMCG",
    "Auto": "^CNXAUTO"
}

In [48]:
def data(index):
    df = yf.download(index, start = "2022-01-01" , end = "2024-12-31")
    df = df.xs(key = index, level='Ticker', axis=1)
    return df

In [49]:
benchmark_data = data("^NSEI")
sector_data = {sector: data(index) for sector, index in sectors.items()}

lookback = 40
dates = benchmark_data.index
rebalance_dates = dates[::lookback]

relative_strength = pd.DataFrame(index = dates, columns=sectors.keys(), dtype=float)
rs_values = {}

for i in range(len(rebalance_dates)):
    date = rebalance_dates[i]
    past_date = date - pd.Timedelta(days=lookback)
    
    if past_date not in benchmark_data.index:
        past_idx = benchmark_data.index.get_indexer([past_date], method='nearest')[0]
        past_date = benchmark_data.index[past_idx]
    if date not in benchmark_data.index:
        date_idx = benchmark_data.index.get_indexer([date], method='nearest')[0]
        date = benchmark_data.index[date_idx]
    rs_row = {}

    for sector, sector_df in sector_data.items():
        if date in sector_df.index and past_date in sector_df.index:
            sector_return = (sector_df.loc[date]['Close'] - sector_df.loc[past_date]['Close']) / sector_df.loc[past_date]['Close']
            benchmark_return = (benchmark_data.loc[date]['Close'] - benchmark_data.loc[past_date]['Close']) / benchmark_data.loc[past_date]['Close']
            rs_row[sector] = sector_return / benchmark_return if benchmark_return != 0 else np.nan

    rs_values[date] = rs_row

rs_dates = sorted(rs_values.keys())
for i in range(len(rs_dates)):
    start_date = rs_dates[i]
    if i + 1 < len(rs_dates):
        end_date = rs_dates[i + 1]
    else:
        end_date = relative_strength.index[-1]
    
    add_dates = (relative_strength.index >= start_date) & (relative_strength.index < end_date)
    for sector in sectors.keys():
        relative_strength.loc[add_dates, sector] = rs_values[start_date].get(sector, np.nan)
        
relative_strength = pd.DataFrame.dropna(relative_strength, axis = 0)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [50]:
top_sector_price = []

for date in relative_strength.index:
    top_sector = relative_strength.loc[date].astype(float).idxmax()
    sector_df = sector_data[top_sector]
    
    if date not in sector_df.index:
        continue
    
    row = sector_df.loc[date][['Open', 'High', 'Low', 'Close', 'Volume']]
    row['Sector'] = top_sector
    top_sector_price.append(row)

df = pd.DataFrame(top_sector_price)

factor = 1000
df.Open /= factor
df.High /= factor
df.Low /= factor
df.Close /= factor

In [51]:
class EMACrossoverADX(Strategy):

    window_time1 = 8
    window_time2 = 40
    timeperiod = 14
    rebalance_period = 40
    
    def init(self):
        self.count = 0
        close = self.data.Close
        high = self.data.High
        low = self.data.Low
        
        self.ema1 = self.I(talib.EMA, close, self.window_time1)
        self.ema2 = self.I(talib.EMA, close, self.window_time2)
        self.adx = self.I(talib.ADX, high, low, close, self.timeperiod)

    def next(self):
        self.count += 1
        if self.count % self.rebalance_period == 0:
            if self.position:
                self.position.close()
        ema1 = self.ema1[-1]
        ema2 = self.ema2[-1]

        if ema1 > ema2 and not self.position and self.adx[-1] > 25:
            self.buy(sl = self.data.Close*0.9)
        elif ema1 < ema2 and self.position:
            self.position.close()
        

In [52]:
bt = Backtest(df, EMACrossoverADX, cash = 100000)
stats = bt.optimize(window_time1 = range(10,20), window_time2 = range(25,40), maximize = 'Sharpe Ratio')
print(stats)
bt.plot()

C:\Users\Aryan jain\Desktop\programs\CFA\bt\Lib\site-packages\backtesting\backtesting.py:1606: RuntimeWarning: If you want to use multi-process optimization with `multiprocessing.get_start_method() == 'spawn'` (e.g. on Windows),set `backtesting.Pool = multiprocessing.Pool` (or of the desired context) and hide `bt.optimize()` call behind a `if __name__ == '__main__'` guard. Currently using thread-based paralellism, which might be slightly slower for non-numpy / non-GIL-releasing code. See https://github.com/kernc/backtesting.py/issues/1256
  output = _optimize_grid()


Start                     2022-03-02 00:00:00
End                       2024-12-27 00:00:00
Duration                   1031 days 00:00:00
Exposure Time [%]                    49.77099
Equity Final [$]                 147318.96397
Equity Peak [$]                  153988.00538
Return [%]                           47.31896
Buy & Hold Return [%]               358.46205
Return (Ann.) [%]                    16.07391
Volatility (Ann.) [%]                18.14627
CAGR [%]                              9.93254
Sharpe Ratio                           0.8858
Sortino Ratio                         1.73067
Calmar Ratio                           1.0721
Alpha [%]                            43.90152
Beta                                  0.00953
Max. Drawdown [%]                   -14.99289
Avg. Drawdown [%]                    -2.64888
Max. Drawdown Duration      311 days 00:00:00
Avg. Drawdown Duration       33 days 00:00:00
# Trades                                   14
Win Rate [%]                      

GridPlot(id='p5234', ...)